## `Questions` we have to answer

- What was the best month for sales? How much was earned that month?
- What city sold the most product?
- What time should we display advertisemens to maximize the likelihood of customer’s buying product?
- What products are most often sold together?
- What product sold the most? Why do you think it sold the most?

# Import `Packages`

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

In [7]:
# for test and viz chart 
#import dtale
#dtale.show(Product_QtOrd, open_browser=True)

### Read all files and concat them in 1 file

In [10]:
path = r'C:\Users\chaib\OneDrive\سطح المكتب\case stady\Finished project\Python\Sales_Data\Sales_Data'
files = [file for file in os.listdir(path) if not file.startswith('.')] # Ignore hidden files

all_months_data = pd.DataFrame()

for file in files:
    current_data = pd.read_csv(path+"/"+file)
    all_months_data = pd.concat([all_months_data, current_data])
    
all_months_data.to_csv("all_data_copy.csv", index=False)

In [12]:
df1 = pd.read_csv("all_data_copy.csv")

#create a copy of the df
df = df1.copy()
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### `Clean` the data

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [14]:
df.isna().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [15]:
df = df.dropna()

In [16]:
df.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [17]:
unique_values = {column: df[column].unique() for column in df.columns}
unique_values

{'Order ID': array(['176558', '176559', '176560', ..., '259355', '259356', '259357'],
       dtype=object),
 'Product': array(['USB-C Charging Cable', 'Bose SoundSport Headphones',
        'Google Phone', 'Wired Headphones', 'Macbook Pro Laptop',
        'Lightning Charging Cable', '27in 4K Gaming Monitor',
        'AA Batteries (4-pack)', 'Apple Airpods Headphones',
        'AAA Batteries (4-pack)', 'iPhone', 'Flatscreen TV',
        '27in FHD Monitor', '20in Monitor', 'LG Dryer', 'ThinkPad Laptop',
        'Vareebadd Phone', 'LG Washing Machine', '34in Ultrawide Monitor',
        'Product'], dtype=object),
 'Quantity Ordered': array(['2', '1', '3', '5', 'Quantity Ordered', '4', '7', '6', '8', '9'],
       dtype=object),
 'Price Each': array(['11.95', '99.99', '600', '11.99', '1700', '14.95', '389.99',
        '3.84', '150', '2.99', '700', '300', '149.99', '109.99', '600.0',
        '999.99', '400', '379.99', 'Price Each', '700.0', '1700.0',
        '150.0', '300.0', '400.0'], dtype=o

### information about the concatenat the CSV files

When concatenating multiple CSV files, it's possible that the headers from all the files get mixed with our data.

In this case, we are checking the `Price Each` and `Quantity Ordered` column because this columns normaly have only numeric value 

In [18]:
alpha_values = set()

# Iterate through the 'Order Date' column
for value in df['Price Each']:
    if any(char.isalpha() for char in value):
        if value not in alpha_values:
            alpha_values.add(value)
            print(value)

Price Each


In [19]:
alpha_values = set()
for value in df['Quantity Ordered']:
    if any(char.isalpha() for char in value):
        if value not in alpha_values:
            alpha_values.add(value)
            print(value)

Quantity Ordered


#### Drop the extra headers that may have been added during concatenation and are not part of the actual data.

In [20]:
## because all the extra header that we want to drop are in the same row every time we can just
## drop one of them to clean the df
df = df[df['Product'] != 'Product']

In [21]:
##test if that work
alpha_values = set()
for value in df['Quantity Ordered']:
    if any(char.isalpha() for char in value):
        if value not in alpha_values:
            alpha_values.add(value)
            print(value)

### `Convert` the columns

In [22]:
# Convert 'Order ID' and 'Quantity Ordered' to integer data type
df[['Order ID', 'Quantity Ordered']] = df[['Order ID', 'Quantity Ordered']].astype('Int64')

# Convert 'Price Each' to float format
df['Price Each'] = df['Price Each'].astype('float')

# Split the 'Order Date' column
df[['Order Date', 'Time']] = df['Order Date'].str.split(' ', n=1, expand=True)
# Convert 'Order Date' column to datetime format with the correct format
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y')


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185950 entries, 0 to 186849
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  Int64         
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  Int64         
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
 6   Time              185950 non-null  object        
dtypes: Int64(2), datetime64[ns](1), float64(1), object(3)
memory usage: 11.7+ MB


In [24]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Time
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001",08:46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215",22:30
3,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14:38
4,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14:38
5,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001",09:27


#### split the `Purchase Address` column to have only the city

In [25]:
# Define a function to extract the city from an address
def get_city(address):
    # Split the address string using commas as the separator, and get the second part
    return address.split(",")[1].strip(" ")

# Define a function to extract the state from an address
def get_state(address):
    # Split the address string using commas as the separator, and get the third part
    # Then split the third part using spaces as the separator and get the second element (the state)
    return address.split(",")[2].split(" ")[1]

# Apply the 'get_city' and 'get_state' functions to the 'Purchase Address' column of the DataFrame
# Combine the city and state information, and store it in a new 'City' column
df['City'] = df['Purchase Address'].apply(lambda x: f"{get_city(x)}  ({get_state(x)})")

# Display the first few rows of the updated DataFrame
df.head()


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Time,City
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001",08:46,Dallas (TX)
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215",22:30,Boston (MA)
3,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14:38,Los Angeles (CA)
4,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",14:38,Los Angeles (CA)
5,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001",09:27,Los Angeles (CA)


In [26]:
#drop the columns that we dont need for our analysis
df = df.drop(columns=['Purchase Address'])

In [27]:
## create a sales columns to answer the first question
df['Sales'] = df['Price Each']*df['Quantity Ordered']

In [28]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Time,City,Sales
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,08:46,Dallas (TX),23.9
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,22:30,Boston (MA),99.99
3,176560,Google Phone,1,600.00,2019-04-12,14:38,Los Angeles (CA),600.0
4,176560,Wired Headphones,1,11.99,2019-04-12,14:38,Los Angeles (CA),11.99
5,176561,Wired Headphones,1,11.99,2019-04-30,09:27,Los Angeles (CA),11.99


#### Task 1 :  `Anwser` the question : 
- What was the best month for sales? How much was earned that month?

In [29]:
# Group the data by 'YearMonth' and calculate the sum of sales for each group
df.groupby(df['Order Date'].dt.to_period('M'))['Sales'].sum().reset_index()

,Order Date,Sales
0,2019-01,1813586.44
1,2019-02,2202022.42
2,2019-03,2807100.38
3,2019-04,3390670.24
4,2019-05,3152606.75
5,2019-06,2577802.26
6,2019-07,2647775.76
7,2019-08,2244467.88
8,2019-09,2097560.13
9,2019-10,3736726.88


From the result above, it appears that `2020-01` has the lowest sales, but the results are too small. I'm going to check if we have all the data for this month.

In [30]:
# check how many days we have in 2020
df[df['Order Date'].dt.year == 2020]['Order Date'].unique()

<DatetimeArray>
['2020-01-01 00:00:00']
Length: 1, dtype: datetime64[ns]

In [31]:
# check how many days we have in 2020
df[df['Order Date'].dt.year == 2020]['Order Date'].dt.date.nunique()

1

It appears that we only have data for a single day in 2020, and this could potentially distort the dataset. Therefore, I am going to delete it.

In [32]:
# delete 2020 data 
df = df[df['Order Date'].dt.year != 2020]

In [33]:
# Group the data by 'YearMonth' and calculate the sum of sales for each group
Months_Sales = df.groupby(df['Order Date'].dt.to_period('M'))['Sales'].sum().reset_index()
Months_Sales

,Order Date,Sales
0,2019-01,1813586.44
1,2019-02,2202022.42
2,2019-03,2807100.38
3,2019-04,3390670.24
4,2019-05,3152606.75
5,2019-06,2577802.26
6,2019-07,2647775.76
7,2019-08,2244467.88
8,2019-09,2097560.13
9,2019-10,3736726.88


In [34]:
Months_Sales['Order Date'] = Months_Sales['Order Date'].dt.to_timestamp()

In [35]:
# Create a line chart
fig = px.line(Months_Sales, x='Order Date', y='Sales', title='Sales Over Time',
              markers=True, line_shape='linear', labels={'Order Date': 'Order Date', 'Sales': 'Sales'})
fig.update_yaxes(range=[0,1.1* Months_Sales['Sales'].max()])  
fig.update_layout(showlegend=False)  

# Create a function to format y-axis labels in millions
def millions_formatter(x):
    return f'{x / 1e6:.0f}M'

# Apply the millions_formatter to the y-axis
fig.update_yaxes(tickvals=[i * 1e6 for i in range(int(Months_Sales['Sales'].max() / 1e6) + 1)],
                 ticktext=[millions_formatter(i * 1e6) for i in range(int(Months_Sales['Sales'].max() / 1e6) + 1)],
                 title_text='Sales', title_standoff=0)

# Show the chart
fig.show()


Looking at the sales chart, it's easy to see that October and December stand out with the highest sales with '3M7' '4M6' Notably, they consistently perform twice as well as January.

with this we answer our first question : 
- What was the best month for sales? How much was earned that month?

#### Task 2 :  `Anwser` the question : 
- What city sold the most product?

In [36]:
df.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Time,City,Sales
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,08:46,Dallas (TX),23.9
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,22:30,Boston (MA),99.99
3,176560,Google Phone,1,600.00,2019-04-12,14:38,Los Angeles (CA),600.0


In [37]:
## check the Quantity ordered for each city
Quantity_ByCity = df.groupby('City')[['Quantity Ordered','Sales']].sum().sort_values("Quantity Ordered")
Quantity_ByCity = Quantity_ByCity.reset_index()
Quantity_ByCity

,City,Quantity Ordered,Sales
0,Portland (ME),2750,449758.27
1,Austin (TX),11151,1819081.77
2,Portland (OR),11301,1870579.35
3,Seattle (WA),16552,2747605.49
4,Atlanta (GA),16602,2795498.58
5,Dallas (TX),16728,2766260.45
6,Boston (MA),22524,3660315.12
7,New York City (NY),27916,4662976.81
8,Los Angeles (CA),33288,5451570.81
9,San Francisco (CA),50226,8259719.03


In [38]:
#plot the Quantity_ByCity df created
fig = px.bar(Quantity_ByCity, x='City', y='Quantity Ordered',
             labels={'Quantity Ordered': 'Quantity Ordered'})

fig.update_xaxes(tickangle=45, tickvals=Quantity_ByCity['City'], tickmode='array', ticktext=Quantity_ByCity['City'])

fig.update_xaxes(title_text='City')
fig.update_yaxes(title_text='Quantity Ordered')

fig.update_layout(title_text='Quantity of Orders by City', title_x=0.5)

fig.show()


As clearly indicated in the data, 'San Francisco (CA)' stands out as the city with the highest quantity of products sold. On the other hand, 'Portland (ME)' has the lowest quantity of products sold.

with this we answer the question : 
- What city sold the most product?

#### Task 3 :  `Anwser` the question : 
- What time should we display advertisemens to maximize the likelihood of customer’s buying product?

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185916 entries, 0 to 186849
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185916 non-null  Int64         
 1   Product           185916 non-null  object        
 2   Quantity Ordered  185916 non-null  Int64         
 3   Price Each        185916 non-null  float64       
 4   Order Date        185916 non-null  datetime64[ns]
 5   Time              185916 non-null  object        
 6   City              185916 non-null  object        
 7   Sales             185916 non-null  Float64       
dtypes: Float64(1), Int64(2), datetime64[ns](1), float64(1), object(3)
memory usage: 13.3+ MB


In [40]:
df.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Time,City,Sales
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,08:46,Dallas (TX),23.9
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,22:30,Boston (MA),99.99
3,176560,Google Phone,1,600.00,2019-04-12,14:38,Los Angeles (CA),600.0


In [41]:
df[['Hour', 'Minute']] = df['Time'].str.split(':', n=1, expand=True)

In [42]:
Sales_ByHour = df.groupby(df['Hour'])[['Quantity Ordered', 'Sales']].sum().reset_index().sort_values('Hour')
Sales_ByHour

,Hour,Quantity Ordered,Sales
0,00,4412,711060.59
1,01,2610,458561.97
2,02,1394,232586.5
3,03,922,144697.99
4,04,933,162396.09
5,05,1491,230564.88
6,06,2810,448113.0
7,07,4556,744854.12
8,08,7002,1192348.97
9,09,9816,1639030.58


In [43]:
if isinstance(Sales_ByHour, (pd.DatetimeIndex, pd.MultiIndex)):
    Sales_ByHour = Sales_ByHour.to_frame(index=False)


Sales_ByHour = Sales_ByHour.reset_index().drop('index', axis=1, errors='ignore')
Sales_ByHour.columns = [str(c) for c in Sales_ByHour.columns] 

chart_data = pd.concat([
    Sales_ByHour['Hour'],
    Sales_ByHour['Quantity Ordered'],
    Sales_ByHour['Sales'],
], axis=1)
chart_data = chart_data.sort_values(['Hour'])
chart_data = chart_data.rename(columns={'Hour': 'x'})
chart_data = chart_data.dropna()

charts = []
line_cfg = {'line': {'shape': 'spline', 'smoothing': 0.3}, 'mode': 'lines'}
charts.append(go.Scatter(
    x=chart_data['x'], y=chart_data['Quantity Ordered'], name='Quantity Ordered', **line_cfg
))
charts.append(go.Scatter(
    x=chart_data['x'], y=chart_data['Sales'], name='Sales', yaxis='y2', **line_cfg
))
figure = go.Figure(data=charts, layout=go.Layout({
    'legend': {'orientation': 'h', 'y': -0.3},
    'title': {'text': 'Quantity Ordered, Sales by Hour'},
    'xaxis': {'title': {'text': 'Hour'}},
    'yaxis': {'range': [None, None], 'tickformat': '0:g', 'title': {'text': 'Quantity Ordered'}, 'type': 'linear'},
    'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right', 'title': {'text': 'Sales'}, 'type': 'linear'}
}))

In [44]:
init_notebook_mode(connected=True)

for chart in charts:
    chart.pop('id', None)

iplot(figure)

Based on the sales data provided, the best times to display advertisements to maximize the likelihood of customers buying products would be around 11, 12 in the morning and 7 (19) in the evening. These are the peak sales hours.

Morning (around 6 AM to 7 AM): Displaying advertisements between 6 AM and 7 AM is a strategic choice. During this time, many people are still at home, possibly having breakfast or preparing for their workday. This early morning slot can capture the attention of individuals before they start their work activities. It's an opportunity to reach them while they are relaxed and potentially interested in your products.

Evening (around 7 PM): Running advertisements from 6:30 PM to 7:30 PM is a good strategy, especially if your target audience includes working individuals. Many people finish their workday around this time, making it an opportune moment to promote your products.

with this we answer the question : 
- What time should we display advertisemens to maximize the likelihood of customer’s buying product?

#### Task 4 :  `Anwser` the question : 
- What products are most often sold together?

In [45]:
duplicate_df = df[df['Order ID'].duplicated(keep=False)].copy()  

# Use .loc to set values in the copy
duplicate_df['grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

# Select columns and drop duplicates
duplicate_df = duplicate_df[['Order ID', 'grouped']].drop_duplicates()
duplicate_df.head()


,Order ID,grouped
3,176560,"Google Phone,Wired Headphones"
18,176574,"Google Phone,USB-C Charging Cable"
30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586,"AAA Batteries (4-pack),Google Phone"
119,176672,"Lightning Charging Cable,USB-C Charging Cable"


In [46]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in duplicate_df['grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

print("The Top 10 of the 2 product that are sold together: ")
print('')
for key,value in count.most_common(10):
    print(key, value)

The Top 10 of the 2 product that are sold together: 

('iPhone', 'Lightning Charging Cable') 1004
('Google Phone', 'USB-C Charging Cable') 987
('iPhone', 'Wired Headphones') 447
('Google Phone', 'Wired Headphones') 414
('Vareebadd Phone', 'USB-C Charging Cable') 361
('iPhone', 'Apple Airpods Headphones') 359
('Google Phone', 'Bose SoundSport Headphones') 220
('USB-C Charging Cable', 'Wired Headphones') 160
('Vareebadd Phone', 'Wired Headphones') 143
('Lightning Charging Cable', 'Wired Headphones') 92


In [47]:
count = Counter()

for row in duplicate_df['grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 3)))

print("The Top 10 of the 3 product that are sold together: ")
print('')
for key,value in count.most_common(10):
    print(key, value)

The Top 10 of the 3 product that are sold together: 

('Google Phone', 'USB-C Charging Cable', 'Wired Headphones') 87
('iPhone', 'Lightning Charging Cable', 'Wired Headphones') 62
('iPhone', 'Lightning Charging Cable', 'Apple Airpods Headphones') 47
('Google Phone', 'USB-C Charging Cable', 'Bose SoundSport Headphones') 35
('Vareebadd Phone', 'USB-C Charging Cable', 'Wired Headphones') 33
('iPhone', 'Apple Airpods Headphones', 'Wired Headphones') 27
('Google Phone', 'Bose SoundSport Headphones', 'Wired Headphones') 24
('Vareebadd Phone', 'USB-C Charging Cable', 'Bose SoundSport Headphones') 16
('USB-C Charging Cable', 'Bose SoundSport Headphones', 'Wired Headphones') 5
('Vareebadd Phone', 'Bose SoundSport Headphones', 'Wired Headphones') 5


From the results above, it appears that customers often purchase specific combinations of products. The most common combination consists of a phone, a charging cable, and a pair of headphones. This pattern suggests that when customers buy a phone, they frequently choose to purchase a charging cable and headphones as well.

This trend might indicate that customers want to ensure they have the necessary accessories to use their new phone effectively. It's a practical choice, as a charging cable and headphones are often essential for a seamless phone experience.

with this we answer the question : 
- What products are most often sold together?

#### Task 5 :  `Anwser` the question : 
- What product sold the most? Why do you think it sold the most?

In [48]:
Product_QtOrd = df.groupby('Product')[['Quantity Ordered','Sales']].sum().sort_values("Quantity Ordered",ascending=False)
Product_QtOrd['Price_AVG'] = df.groupby('Product')[('Price Each')].mean()
Product_QtOrd = Product_QtOrd.reset_index()
Product_QtOrd

,Product,Quantity Ordered,Sales,Price_AVG
0,AAA Batteries (4-pack),31012,92725.88,2.99
1,AA Batteries (4-pack),27635,106118.4,3.84
2,USB-C Charging Cable,23971,286453.45,11.95
3,Lightning Charging Cable,23211,347004.45,14.95
4,Wired Headphones,20553,246430.47,11.99
5,Apple Airpods Headphones,15657,2348550.0,150.00
6,Bose SoundSport Headphones,13454,1345265.46,99.99
7,27in FHD Monitor,7547,1131974.53,149.99
8,iPhone,6847,4792900.0,700.00
9,27in 4K Gaming Monitor,6243,2434707.57,389.99


In [49]:
# the code is not small but simple it only create 2 chart one above the others 

#Define a function to create a product chart
def create_product_chart(x, y1, y2, y1_name, y2_name, x_title, y1_title, y2_title, chart_title):
    # Create a subplot with dual y-axes
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add a line chart (scatter plot) for the first data series (y1)
    fig.add_trace(
        go.Scatter(x=x, y=y1, name=y1_name, mode="lines"),
        secondary_y=True
    )

    # Add a bar chart for the second data series (y2)
    fig.add_trace(
        go.Bar(x=x, y=y2, name=y2_name),
        secondary_y=False
    )

    # Update the x-axis label
    fig.update_xaxes(title_text=x_title)

    # Update the y-axis labels for the two data series
    fig.update_yaxes(title_text=y1_title, secondary_y=True)
    fig.update_yaxes(title_text=y2_title, secondary_y=False)

    # Set the chart title
    fig.update_layout(title_text=chart_title, title_x=0.5)

    # Display the chart
    fig.show()

# Create the first chart: Sales and Quantity Ordered by Product
create_product_chart(
    x=Product_QtOrd['Product'],
    y1=Product_QtOrd['Sales'],
    y2=Product_QtOrd['Quantity Ordered'],
    y1_name="Sales",
    y2_name="Quantity",
    x_title="Product",
    y1_title="Sales",
    y2_title="Quantity Ordered",
    chart_title="Sales and Quantity Ordered by Product"
)

# Create the second chart: Price and Quantity Ordered by Product
create_product_chart(
    x=Product_QtOrd['Product'],
    y1=Product_QtOrd['Price_AVG'],
    y2=Product_QtOrd['Quantity Ordered'],
    y1_name="Price",
    y2_name="Quantity",
    x_title="Product",
    y1_title="Price",
    y2_title="Quantity Ordered",
    chart_title="Price and Quantity Ordered by Product"
)



The product that sold the most in the chart  is the AAA Batteries (4-pack), We have 31012 Quantity ordered. I think there are a few reasons why this product sold the most:

- Affordability: AAA batteries are a relatively inexpensive product, making them accessible to a wide range of consumers.
- Necessity: AAA batteries are used in a wide variety of devices, including remote controls, toys, flashlights, and many more. This makes them an essential household item for many people.
- Convenience: AAA batteries are widely available at most stores, making them easy to purchase when needed.

with this we Anwser the question : 
- What product sold the most? Why do you think it sold the most?

#### Task 6 :  dive bit more in the data : 
- Analysis of Customer Purchasing Habits by Day of the Week ?

In [50]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Time,City,Sales,Hour,Minute
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,08:46,Dallas (TX),23.9,08,46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,22:30,Boston (MA),99.99,22,30
3,176560,Google Phone,1,600.00,2019-04-12,14:38,Los Angeles (CA),600.0,14,38
4,176560,Wired Headphones,1,11.99,2019-04-12,14:38,Los Angeles (CA),11.99,14,38
5,176561,Wired Headphones,1,11.99,2019-04-30,09:27,Los Angeles (CA),11.99,09,27


In [51]:
df['day_of_week'] = df['Order Date'].dt.day_name()

In [52]:
df["is_weekend"] = np.where((df['day_of_week'] == 'Friday') | (df['day_of_week'] == 'Saturday'), 1, 0)

In [53]:
week = df.groupby(df['day_of_week'])[['Quantity Ordered', 'Sales']].sum().reset_index()
week

,day_of_week,Quantity Ordered,Sales
0,Friday,29448,4855938.38
1,Monday,29899,4883326.72
2,Saturday,29769,4904357.01
3,Sunday,29778,4932169.66
4,Thursday,29662,4839465.16
5,Tuesday,30724,5087956.78
6,Wednesday,29758,4980151.97


In [54]:
# Define the desired order of days of the week
custom_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create a Categorical data type with the custom order
cat_dtype = pd.CategoricalDtype(categories=custom_order, ordered=True)

# Apply the custom data type to the 'day_of_week' column
week ['day_of_week'] = week ['day_of_week'].astype(cat_dtype)

# Sort the DataFrame by the 'day_of_week' column
week = week.sort_values('day_of_week')

# Reset the index to match the desired format
week.reset_index(drop=True, inplace=True)

In [55]:
week.head(7)

,day_of_week,Quantity Ordered,Sales
0,Monday,29899,4883326.72
1,Tuesday,30724,5087956.78
2,Wednesday,29758,4980151.97
3,Thursday,29662,4839465.16
4,Friday,29448,4855938.38
5,Saturday,29769,4904357.01
6,Sunday,29778,4932169.66


In [56]:
if isinstance(week, (pd.DatetimeIndex, pd.MultiIndex)):
    week = week.to_frame(index=False)


week = week.reset_index().drop('index', axis=1, errors='ignore')
week.columns = [str(c) for c in week.columns]  

chart_data = pd.concat([
    week['day_of_week'],  
    week['Quantity Ordered'],   
    week['Sales'],   
], axis=1)

chart_data = chart_data.sort_values(['day_of_week'])
chart_data = chart_data.rename(columns={'day_of_week': 'x'})
chart_data = chart_data.dropna()

charts = []
line_cfg = {'line': {'shape': 'spline', 'smoothing': 0.3}, 'mode': 'lines'}
charts.append(go.Scatter(
    x=chart_data['x'], y=chart_data['Quantity Ordered'], name='Quantity Ordered', **line_cfg
))
charts.append(go.Scatter(
    x=chart_data['x'], y=chart_data['Sales'], name='Sales', yaxis='y2', **line_cfg
))
figure = go.Figure(data=charts, layout=go.Layout({
    'legend': {'orientation': 'h', 'y': -0.3},
    'title': {'text': 'Quantity Ordered, Sales by day_of_week'},
    'xaxis': {'title': {'text': 'day_of_week'}},
    'yaxis': {'range': [0, 30724], 'tickformat': '0:g', 'title': {'text': 'Quantity Ordered'}, 'type': 'linear'},
    'yaxis2': {'anchor': 'x',
               'overlaying': 'y',
               'range': [0, 5087956.78],
               'side': 'right',
               'title': {'text': 'Sales'},
               'type': 'linear'}
}))


In [57]:
init_notebook_mode(connected=True)

for chart in charts:
    chart.pop('id', None)

iplot(figure)

##### Analysis of Customer Purchasing Habits by Day of the Week
After a thorough analysis of the provided data, it appears that the differences in customer purchasing habits based on the day of the week are relatively small.
This suggests that customers tend to buy our products consistently throughout the week, with only minor fluctuations in order quantities and sales figures from Monday to Sunday.


### `Sales Analysis and Insights`



`Best Month for Sales and Earnings`: October and December were the best months for sales, with earnings of '3M7' and '4M6,' respectively. These months consistently outperformed January.

`City with Most Product Sales`: 'San Francisco (CA)' stands out as the city with the highest quantity of products sold, while 'Portland (ME)' had the lowest quantity of products sold.

`Optimal Advertisement Times`: To maximize the likelihood of customer purchases, consider displaying advertisements around 11 AM, 12 PM, and 7 PM. These times correspond to peak sales hours when customers are more likely to buy products.

`Frequently Sold Together Products`: Customers often purchase a combination of a phone, charging cable, and headphones. This indicates that when customers buy a phone, they tend to buy these accessories for a seamless experience.

`Best-Selling Product`: The product that sold the most is the AAA Batteries (4-pack) with 31,012 quantity ordered. This can be attributed to its affordability, necessity, and convenience for consumers.

`Analysis of Customer Purchasing Habits by Day of the Week`: Customer purchasing habits show relatively small differences throughout the week. This suggests consistent sales across weekdays and weekends.